# Data Import & Connection to BigQuery

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from google.cloud import bigquery
from itertools import product
from functions import *

# set display options to show all columns
pd.set_option('display.max_columns', None)
# Set the float format to display numbers without scientific notation
pd.options.display.float_format = '{:.2f}'.format
# Set the client for future queries to BigQuery
client = bigquery.Client(project = "continente-lced-feup")

C:\Users\luish\anaconda3\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=nncOts8YxzC8jHgBu0YQGd2KEI4xbH&access_type=offline&code_challenge=0OHVxkxFROkqnywJ1MTdDkvwo0aWSI8CHkTaR3cswRE&code_challenge_method=S256


Credentials saved to file: [C:\Users\luish\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot find a quota project to add to ADC. You might receive a "quota exceeded" or "API not enabled" error. Run $ gcloud auth application-default set-quota-project to add a 

# Data Loading

In [49]:
query = client.query("""
   SELECT *
   FROM 
       tables_raw.dim_customer 
       JOIN tables_raw.fact_transaction USING(CUSTOMER_ACCOUNT_NR_MASK)
       JOIN tables_raw.dim_product USING(SKU)
    WHERE 
        SUBCAT_CD_EXT IN (140304, 50401, 30301, 20201, 10301, 80103, 60102, 60401, 30401, 10101, 100102, 100204, 140204, 
        20302, 30201, 50203, 90201, 170101, 80404, 50402, 70202, 100201, 80409, 10303, 20306, 80411, 10102, 20305, 60105, 80110, 
        140301, 30202, 90202, 100101, 80105, 80104, 50202, 50303, 70204, 60306, 80403, 10302, 10201, 80405, 170304, 170303, 80406, 
        140201, 60302, 30403, 30304, 20204, 170106, 140205, 10204, 60404, 50301, 50302, 20205, 60406, 20301, 80407, 20203, 70201, 100205,
        60106, 170302, 50201, 60301, 10205, 30203, 80401, 100202, 30302, 170111, 10202, 70203, 60303, 170109, 60403, 30402, 140302, 30208, 60307, 
        80107, 50403, 60103, 20307, 60305, 60101, 170307, 80414, 80415, 60405, 20303, 80402, 30204, 30206, 170310, 60304, 140206, 10203, 30205, 60107, 
        70206, 170108, 90203, 90204, 30207, 140303, 30303, 80408, 140202, 50304, 80101, 170313, 100203, 60402, 170305, 50305, 50404, 20202, 170110, 
        170105, 170112, 170301, 10206, 10208, 20304, 80102, 70205, 10207, 10305, 170309, 170114, 80111, 90206, 30306, 30305, 140203, 80413) 
        AND SEG_AGE_DSC = ']25;35]'
        AND QTY >= 0
    ORDER BY CUSTOMER_ACCOUNT_NR_MASK DESC, TIME_KEY ASC
   """)

df = query.result().to_dataframe() # Wait for the job to complete.

# Data Preparation (more to be done...)

In [50]:
df = df.drop(columns=['LOC_BRAND_CD','PROD_DSCNT_ISSUED_AMT','NET_SLS_AMT','TRANS_DSCNT_RAT_AMT','DIRECT_DSCNT_AMT',
                 'seg_lifestyle_dsc','SEG_AGE','SEG_AGE_DSC','seg_lifestage_dsc',
                 'UNIT_BASE_DSC_EXT','SUBCAT_DSC_EXT','BIZ_UNIT_DSC_EXT','DEPARTMENT_DSC_EXT',
                'PRODUCT_SHORT_DSC','BRAND_DSC','BRAND_TYPE_CD','CONVERSION_FACTOR','CAPACITY_UNIT','PRODUCT_DSC','SKU',
                'LOCATION_CD','GROSS_SLS_AMT','CP4','CAT_DSC_EXT','PRODUCT_KEY','POS_TP_CD','PRICE_RANGE'])

In [51]:
df.isnull().sum()

CUSTOMER_ACCOUNT_NR_MASK         0
GENDER                      207890
FAMILY_MEMBERS              778011
seg_lifestyle_cd                 0
seg_lifestage_cd                 0
TIME_KEY                         0
TRANSACTION_ID_MASK              0
QTY                              0
UNIT_BASE_CD_EXT                 0
SUBCAT_CD_EXT                    0
CAT_CD_EXT                       0
BIZ_UNIT_CD_EXT                  0
DEPARTMENT_CD_EXT                0
dtype: int64

# Feature Engineering & ML Dataset Development

In [52]:
# convert the 'TIME_KEY' column to datetime format
df['TIME_KEY'] = pd.to_datetime(df['TIME_KEY'], format='%Y%m%d')

# create new columns for the day, week, day of the week, month, quarter, and year
#df['DAY'] = df['TIME_KEY'].dt.day
#df['WEEK'] = df['TIME_KEY'].dt.week
#df['DOW'] = df['TIME_KEY'].dt.dayofweek
df['MONTH'] = df['TIME_KEY'].dt.month
df['QUARTER'] = df['TIME_KEY'].dt.quarter
df['SEMESTER'] = df['MONTH'].apply(semester)
df['YEAR'] = df['TIME_KEY'].dt.year

In [53]:
# Filter the dataframe by the customer's frequency score
df = filter_customers(df)
# Sort the dataframe (later suffled again)
df = df.sort_values(['TIME_KEY','TRANSACTION_ID_MASK','YEAR','MONTH'])

In [54]:
# Step 2: Drop duplicates, keeping only the first occurrence of each customer ID
df_first_transaction = df.drop_duplicates('CUSTOMER_ACCOUNT_NR_MASK', keep='first')

# Step 3: Create a new DataFrame with customer ID and first transaction date
df_first_transaction = df_first_transaction[['CUSTOMER_ACCOUNT_NR_MASK', 'TIME_KEY']]

# Convert the 'TIME_KEY' column to datetime type
df_first_transaction['TIME_KEY'] = pd.to_datetime(df_first_transaction['TIME_KEY'])

# Extract the year and month from the 'TIME_KEY' column and format it as "year-month"
df_first_transaction['TIME_KEY'] = df_first_transaction['TIME_KEY'].dt.strftime('%Y-%m')

df_first_transaction = df_first_transaction.sort_values(['CUSTOMER_ACCOUNT_NR_MASK'])

In [9]:
# get all unique values of customer_id, category_id, month and year
customer_ids = df['CUSTOMER_ACCOUNT_NR_MASK'].unique()
category_ids = df['SUBCAT_CD_EXT'].unique()
months = df['MONTH'].unique()
years = df['YEAR'].unique()

# create a new dataframe with all possible combinations of customer_id and category_id
ml_dataset = pd.DataFrame(list(product(customer_ids, category_ids, months, years)), 
                                    columns=['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT','MONTH','YEAR'])

# add the quarter and semester columns based on the month value
quarter_map = {1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 2, 7: 3, 8: 3, 9: 3, 10: 4, 11: 4, 12: 4}
semester_map = {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2}

ml_dataset['QUARTER'] = ml_dataset['MONTH'].map(quarter_map)
ml_dataset['SEMESTER'] = ml_dataset['MONTH'].map(semester_map)

In [10]:
# Combine the 'year' and 'month' columns into a single column
ml_dataset['TIME_KEY_agg'] = ml_dataset['YEAR'].astype(str) + '-' + ml_dataset['MONTH'].astype(str)

# Convert the 'TIME_KEY' column to datetime format
ml_dataset['TIME_KEY_agg'] = pd.to_datetime(ml_dataset['TIME_KEY_agg'], format='%Y-%m')

# Extract the year and month from the 'TIME_KEY' column and format it as "year-month"
ml_dataset['TIME_KEY_agg'] = ml_dataset['TIME_KEY_agg'].dt.strftime('%Y-%m')

# Step 3: Merge with the aggregated dataset
ml_dataset = pd.merge(ml_dataset, df_first_transaction, on='CUSTOMER_ACCOUNT_NR_MASK', how='left')

# Step 4: Filter the aggregated dataset based on the first transaction date
ml_dataset = ml_dataset[ml_dataset['TIME_KEY_agg'] >= ml_dataset['TIME_KEY']]

ml_dataset = ml_dataset.drop(columns=['TIME_KEY_agg','TIME_KEY'])

In [55]:
ml_dataset = ml_dataset.sort_values(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT','YEAR','MONTH'])

In [56]:
df = df.sort_values(['CUSTOMER_ACCOUNT_NR_MASK','TIME_KEY'])
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

In [57]:
# TOTAL NUMBER OF ORDERS - CORRETO!

df_uniques = df.drop_duplicates(subset='TRANSACTION_ID_MASK', keep='first').reset_index()

df_uniques['CUST_NUM_TRANSACTIONS_30_DAYS'] = df_uniques.groupby('CUSTOMER_ACCOUNT_NR_MASK').rolling(window='30D', on='TIME_KEY', min_periods=1).TRANSACTION_ID_MASK \
                                                        .count().reset_index()['TRANSACTION_ID_MASK'].astype(int)

df_uniques['CUST_NUM_TRANSACTIONS_90_DAYS'] = df_uniques.groupby('CUSTOMER_ACCOUNT_NR_MASK').rolling(window='90D', on='TIME_KEY', min_periods=1).TRANSACTION_ID_MASK \
                                                        .count().reset_index()['TRANSACTION_ID_MASK'].astype(int)

df_uniques['CUST_NUM_TRANSACTIONS_180_DAYS'] = df_uniques.groupby('CUSTOMER_ACCOUNT_NR_MASK').rolling(window='180D', on='TIME_KEY', min_periods=1).TRANSACTION_ID_MASK \
                                                         .count().reset_index()['TRANSACTION_ID_MASK'].astype(int)

df_uniques['CUST_NUM_TRANSACTIONS_360_DAYS'] = df_uniques.groupby('CUSTOMER_ACCOUNT_NR_MASK').rolling(window='360D', on='TIME_KEY', min_periods=1).TRANSACTION_ID_MASK \
                                                         .count().reset_index()['TRANSACTION_ID_MASK'].astype(int)

df = pd.merge(df, df_uniques[['TRANSACTION_ID_MASK','CUST_NUM_TRANSACTIONS_30_DAYS','CUST_NUM_TRANSACTIONS_90_DAYS','CUST_NUM_TRANSACTIONS_180_DAYS',
                             'CUST_NUM_TRANSACTIONS_360_DAYS']], on=['TRANSACTION_ID_MASK'], how='left')

ml_dataset = create_aggregations(df_uniques, ml_dataset, '30_DAYS', 'CUST_NUM_TRANSACTIONS', 'CUST', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '90_DAYS', 'CUST_NUM_TRANSACTIONS', 'CUST', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '180_DAYS', 'CUST_NUM_TRANSACTIONS', 'CUST', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '360_DAYS', 'CUST_NUM_TRANSACTIONS', 'CUST', False)

"\nml_dataset = create_aggregations(df_uniques, ml_dataset, '30_DAYS', 'CUST_NUM_TRANSACTIONS', 'CUST', False)\nml_dataset = create_aggregations(df_uniques, ml_dataset, '90_DAYS', 'CUST_NUM_TRANSACTIONS', 'CUST', False)\nml_dataset = create_aggregations(df_uniques, ml_dataset, '180_DAYS', 'CUST_NUM_TRANSACTIONS', 'CUST', False)\nml_dataset = create_aggregations(df_uniques, ml_dataset, '360_DAYS', 'CUST_NUM_TRANSACTIONS', 'CUST', False)\n"

In [14]:
# TOTAL QUATITY BOUGHT BY CUSTOMER - CORRETO!

df['CUST_TOTAL_QTY_BOUGHT_30_DAYS'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK').rolling(window='30D', on='TIME_KEY', min_periods=1).QTY \
                                        .sum().reset_index()['QTY'].astype(int)

df['CUST_TOTAL_QTY_BOUGHT_90_DAYS'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK').rolling(window='90D', on='TIME_KEY', min_periods=1).QTY \
                                        .sum().reset_index()['QTY'].astype(int)

df['CUST_TOTAL_QTY_BOUGHT_180_DAYS'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK').rolling(window='180D', on='TIME_KEY', min_periods=1).QTY \
                                        .sum().reset_index()['QTY'].astype(int)

df['CUST_TOTAL_QTY_BOUGHT_360_DAYS'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK').rolling(window='360D', on='TIME_KEY', min_periods=1).QTY \
                                        .sum().reset_index()['QTY'].astype(int)


ml_dataset = create_aggregations(df, ml_dataset, '30_DAYS', 'CUST_TOTAL_QTY_BOUGHT', 'CUST', False)
ml_dataset = create_aggregations(df, ml_dataset, '90_DAYS', 'CUST_TOTAL_QTY_BOUGHT', 'CUST', False)
ml_dataset = create_aggregations(df, ml_dataset, '180_DAYS', 'CUST_TOTAL_QTY_BOUGHT', 'CUST', False)
ml_dataset = create_aggregations(df, ml_dataset, '360_DAYS', 'CUST_TOTAL_QTY_BOUGHT', 'CUST', False)

In [15]:
# UNIQUE NUMBER OF SUBCATEGORIES BOUGHT - CORRETO!

df['CUST_NUM_UNIQUE_SUBCAT_30_DAYS'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK').rolling(window='30D', on='TIME_KEY', min_periods=1).SUBCAT_CD_EXT \
                                        .apply(lambda x: x.nunique()).reset_index()['SUBCAT_CD_EXT'].astype(int)

df['CUST_NUM_UNIQUE_SUBCAT_90_DAYS'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK').rolling(window='90D', on='TIME_KEY', min_periods=1).SUBCAT_CD_EXT \
                                        .apply(lambda x: x.nunique()).reset_index()['SUBCAT_CD_EXT'].astype(int)

df['CUST_NUM_UNIQUE_SUBCAT_180_DAYS'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK').rolling(window='180D', on='TIME_KEY', min_periods=1).SUBCAT_CD_EXT \
                                        .apply(lambda x: x.nunique()).reset_index()['SUBCAT_CD_EXT'].astype(int)

df['CUST_NUM_UNIQUE_SUBCAT_360_DAYS'] = df.groupby('CUSTOMER_ACCOUNT_NR_MASK').rolling(window='360D', on='TIME_KEY', min_periods=1).SUBCAT_CD_EXT \
                                        .apply(lambda x: x.nunique()).reset_index()['SUBCAT_CD_EXT'].astype(int)


ml_dataset = create_aggregations(df, ml_dataset, '30_DAYS', 'CUST_NUM_UNIQUE_SUBCAT', 'CUST', False)
ml_dataset = create_aggregations(df, ml_dataset, '90_DAYS', 'CUST_NUM_UNIQUE_SUBCAT', 'CUST', False)
ml_dataset = create_aggregations(df, ml_dataset, '180_DAYS', 'CUST_NUM_UNIQUE_SUBCAT', 'CUST', False)
ml_dataset = create_aggregations(df, ml_dataset, '360_DAYS', 'CUST_NUM_UNIQUE_SUBCAT', 'CUST', False)

In [16]:
# AVERAGE DAYS FOR NEXT CUSTOMER'S TRANSACTION - CORRETO!

df_uniques = df.copy()
df_uniques['DAYS_FOR_NEXT_TRANSACTION'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK'])['TIME_KEY'].diff(-1).dt.days \
                                              .fillna(0).apply(lambda x: abs(x))
df_uniques = df_uniques.drop_duplicates(subset='TRANSACTION_ID_MASK', keep='last').reset_index()

df_uniques['CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS'] = df_uniques.groupby('CUSTOMER_ACCOUNT_NR_MASK') \
                                                                   .rolling(window='30D', on='TIME_KEY', min_periods=1).DAYS_FOR_NEXT_TRANSACTION \
                                                                   .apply(lambda x: x[:-1].mean()).reset_index()['DAYS_FOR_NEXT_TRANSACTION'] \
                                                                   .fillna(30)

df_uniques['CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS'] = df_uniques.groupby('CUSTOMER_ACCOUNT_NR_MASK') \
                                                                   .rolling(window='90D', on='TIME_KEY', min_periods=1).DAYS_FOR_NEXT_TRANSACTION \
                                                                   .apply(lambda x: x[:-1].mean()).reset_index()['DAYS_FOR_NEXT_TRANSACTION'] \
                                                                   .fillna(60)

df_uniques['CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS'] = df_uniques.groupby('CUSTOMER_ACCOUNT_NR_MASK') \
                                                                    .rolling(window='180D', on='TIME_KEY', min_periods=1).DAYS_FOR_NEXT_TRANSACTION \
                                                                    .apply(lambda x: x[:-1].mean()).reset_index()['DAYS_FOR_NEXT_TRANSACTION'] \
                                                                    .fillna(180)

df_uniques['CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS'] = df_uniques.groupby('CUSTOMER_ACCOUNT_NR_MASK') \
                                                                    .rolling(window='360D', on='TIME_KEY', min_periods=1).DAYS_FOR_NEXT_TRANSACTION \
                                                                    .apply(lambda x: x[:-1].mean()).reset_index()['DAYS_FOR_NEXT_TRANSACTION'] \
                                                                    .fillna(360)

ml_dataset = create_aggregations(df_uniques, ml_dataset, '30_DAYS', 'CUST_AVG_DAYS_FOR_NEXT_TRANSACTION', 'CUST', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '90_DAYS', 'CUST_AVG_DAYS_FOR_NEXT_TRANSACTION', 'CUST', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '180_DAYS', 'CUST_AVG_DAYS_FOR_NEXT_TRANSACTION', 'CUST', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '360_DAYS', 'CUST_AVG_DAYS_FOR_NEXT_TRANSACTION', 'CUST', False)

# Regression Feature - Average days for the next transaction in the next 30 days of a certain month, by customer

ml_dataset['REG_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS'] = ml_dataset.groupby('CUSTOMER_ACCOUNT_NR_MASK') \
                                                                    .CUST_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS \
                                                                    .shift(-1)

In [58]:
# AVERAGE BASKET SIZE - CORRETO!

df['CUST_NUM_SUBCAT_30_DAYS'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK']) \
                                  .rolling(window='30D', on='TIME_KEY', min_periods=1).SUBCAT_CD_EXT \
                                  .count().reset_index()['SUBCAT_CD_EXT']
df['CUST_AVG_BASKET_SIZE_30_DAYS'] = df['CUST_NUM_SUBCAT_30_DAYS'] / df['CUST_NUM_TRANSACTIONS_30_DAYS']

df['CUST_NUM_SUBCAT_90_DAYS'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK']) \
                                  .rolling(window='90D', on='TIME_KEY', min_periods=1).SUBCAT_CD_EXT \
                                  .count().reset_index()['SUBCAT_CD_EXT']
df['CUST_AVG_BASKET_SIZE_90_DAYS'] = df['CUST_NUM_SUBCAT_90_DAYS'] / df['CUST_NUM_TRANSACTIONS_90_DAYS']

df['CUST_NUM_SUBCAT_180_DAYS'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK']) \
                                  .rolling(window='180D', on='TIME_KEY', min_periods=1).SUBCAT_CD_EXT \
                                  .count().reset_index()['SUBCAT_CD_EXT']
df['CUST_AVG_BASKET_SIZE_180_DAYS'] = df['CUST_NUM_SUBCAT_180_DAYS'] / df['CUST_NUM_TRANSACTIONS_180_DAYS']

df['CUST_NUM_SUBCAT_360_DAYS'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK']) \
                                  .rolling(window='360D', on='TIME_KEY', min_periods=1).SUBCAT_CD_EXT \
                                  .count().reset_index()['SUBCAT_CD_EXT']
df['CUST_AVG_BASKET_SIZE_360_DAYS'] = df['CUST_NUM_SUBCAT_360_DAYS'] / df['CUST_NUM_TRANSACTIONS_360_DAYS']

ml_dataset = create_aggregations(df, ml_dataset, '30_DAYS', 'CUST_AVG_BASKET_SIZE', 'CUST', True)
ml_dataset = create_aggregations(df, ml_dataset, '90_DAYS', 'CUST_AVG_BASKET_SIZE', 'CUST', True)
ml_dataset = create_aggregations(df, ml_dataset, '180_DAYS', 'CUST_AVG_BASKET_SIZE', 'CUST', True)
ml_dataset = create_aggregations(df, ml_dataset, '360_DAYS', 'CUST_AVG_BASKET_SIZE', 'CUST', True)

MemoryError: Unable to allocate 3.31 GiB for an array with shape (40, 22197207) and data type int32

Subcategory features

In [17]:
df = df.sort_values(['SUBCAT_CD_EXT','TIME_KEY'])
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

In [18]:
# TOTAL NUMBER OF ORDERS - CORRETO!

df_uniques = df.drop_duplicates(subset='TRANSACTION_ID_MASK', keep='first').reset_index()

df_uniques['SUBCAT_NUM_TRANSACTIONS_30_DAYS'] = df_uniques.groupby('SUBCAT_CD_EXT').rolling(window='30D', on='TIME_KEY', min_periods=1).TRANSACTION_ID_MASK \
                                                        .count().reset_index()['TRANSACTION_ID_MASK'].astype(int)

df_uniques['SUBCAT_NUM_TRANSACTIONS_90_DAYS'] = df_uniques.groupby('SUBCAT_CD_EXT').rolling(window='90D', on='TIME_KEY', min_periods=1).TRANSACTION_ID_MASK \
                                                        .count().reset_index()['TRANSACTION_ID_MASK'].astype(int)

df_uniques['SUBCAT_NUM_TRANSACTIONS_180_DAYS'] = df_uniques.groupby('SUBCAT_CD_EXT').rolling(window='180D', on='TIME_KEY', min_periods=1).TRANSACTION_ID_MASK \
                                                         .count().reset_index()['TRANSACTION_ID_MASK'].astype(int)

df_uniques['SUBCAT_NUM_TRANSACTIONS_360_DAYS'] = df_uniques.groupby('SUBCAT_CD_EXT').rolling(window='360D', on='TIME_KEY', min_periods=1).TRANSACTION_ID_MASK \
                                                         .count().reset_index()['TRANSACTION_ID_MASK'].astype(int)

ml_dataset = create_aggregations(df_uniques, ml_dataset, '30_DAYS', 'SUBCAT_NUM_TRANSACTIONS', 'SUBCAT', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '90_DAYS', 'SUBCAT_NUM_TRANSACTIONS', 'SUBCAT', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '180_DAYS', 'SUBCAT_NUM_TRANSACTIONS', 'SUBCAT', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '360_DAYS', 'SUBCAT_NUM_TRANSACTIONS', 'SUBCAT', False)

In [19]:
# TOTAL QUATITY BOUGHT BY SUBCATEGORY - CORRETO!

df['SUBCAT_TOTAL_QTY_BOUGHT_30_DAYS'] = df.groupby('SUBCAT_CD_EXT').rolling(window='30D', on='TIME_KEY', min_periods=1).QTY \
                                        .sum().reset_index()['QTY'].astype(int)

df['SUBCAT_TOTAL_QTY_BOUGHT_90_DAYS'] = df.groupby('SUBCAT_CD_EXT').rolling(window='90D', on='TIME_KEY', min_periods=1).QTY \
                                        .sum().reset_index()['QTY'].astype(int)

df['SUBCAT_TOTAL_QTY_BOUGHT_180_DAYS'] = df.groupby('SUBCAT_CD_EXT').rolling(window='180D', on='TIME_KEY', min_periods=1).QTY \
                                        .sum().reset_index()['QTY'].astype(int)

df['SUBCAT_TOTAL_QTY_BOUGHT_360_DAYS'] = df.groupby('SUBCAT_CD_EXT').rolling(window='360D', on='TIME_KEY', min_periods=1).QTY \
                                        .sum().reset_index()['QTY'].astype(int)

ml_dataset = create_aggregations(df, ml_dataset, '30_DAYS', 'SUBCAT_TOTAL_QTY_BOUGHT', 'SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, '90_DAYS', 'SUBCAT_TOTAL_QTY_BOUGHT', 'SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, '180_DAYS', 'SUBCAT_TOTAL_QTY_BOUGHT', 'SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, '360_DAYS', 'SUBCAT_TOTAL_QTY_BOUGHT', 'SUBCAT', False)

In [20]:
# UNIQUE NUMBER OF CUSTOMERS WHO BOUGHT FROM A SUBCATEGORY - CORRETO!

df['SUBCAT_NUM_UNIQUE_CUST_30_DAYS'] = df.groupby('SUBCAT_CD_EXT').rolling(window='30D', on='TIME_KEY', min_periods=1).CUSTOMER_ACCOUNT_NR_MASK \
                                         .apply(lambda x: x.nunique()).reset_index()['CUSTOMER_ACCOUNT_NR_MASK'].astype(int)

df['SUBCAT_NUM_UNIQUE_CUST_90_DAYS'] = df.groupby('SUBCAT_CD_EXT').rolling(window='90D', on='TIME_KEY', min_periods=1).CUSTOMER_ACCOUNT_NR_MASK \
                                         .apply(lambda x: x.nunique()).reset_index()['CUSTOMER_ACCOUNT_NR_MASK'].astype(int)

df['SUBCAT_NUM_UNIQUE_CUST_180_DAYS'] = df.groupby('SUBCAT_CD_EXT').rolling(window='180D', on='TIME_KEY', min_periods=1).CUSTOMER_ACCOUNT_NR_MASK \
                                         .apply(lambda x: x.nunique()).reset_index()['CUSTOMER_ACCOUNT_NR_MASK'].astype(int)

df['SUBCAT_NUM_UNIQUE_CUST_360_DAYS'] = df.groupby('SUBCAT_CD_EXT').rolling(window='360D', on='TIME_KEY', min_periods=1).CUSTOMER_ACCOUNT_NR_MASK \
                                         .apply(lambda x: x.nunique()).reset_index()['CUSTOMER_ACCOUNT_NR_MASK'].astype(int)

ml_dataset = create_aggregations(df, ml_dataset, '30_DAYS', 'SUBCAT_NUM_UNIQUE_CUST', 'SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, '90_DAYS', 'SUBCAT_NUM_UNIQUE_CUST', 'SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, '180_DAYS', 'SUBCAT_NUM_UNIQUE_CUST', 'SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, '360_DAYS', 'SUBCAT_NUM_UNIQUE_CUST', 'SUBCAT', False)

Customer-Subcategory features

In [21]:
df = df.sort_values(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT','TIME_KEY'])
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

In [22]:
# TOTAL NUMBER OF ORDERS FOR A SUBCATEGORY BY A SPECIFIC CUSTOMER - CORRETO!

df_uniques = df.drop_duplicates(subset='TRANSACTION_ID_MASK', keep='first').reset_index()

df_uniques['CUSTSUBCAT_NUM_TRANSACTIONS_30_DAYS'] = df_uniques.groupby(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT']).rolling(window='30D', on='TIME_KEY', min_periods=1).TRANSACTION_ID_MASK \
                                                        .count().reset_index()['TRANSACTION_ID_MASK'].astype(int)

df_uniques['CUSTSUBCAT_NUM_TRANSACTIONS_90_DAYS'] = df_uniques.groupby(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT']).rolling(window='90D', on='TIME_KEY', min_periods=1).TRANSACTION_ID_MASK \
                                                        .count().reset_index()['TRANSACTION_ID_MASK'].astype(int)

df_uniques['CUSTSUBCAT_NUM_TRANSACTIONS_180_DAYS'] = df_uniques.groupby(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT']).rolling(window='180D', on='TIME_KEY', min_periods=1).TRANSACTION_ID_MASK \
                                                         .count().reset_index()['TRANSACTION_ID_MASK'].astype(int)

df_uniques['CUSTSUBCAT_NUM_TRANSACTIONS_360_DAYS'] = df_uniques.groupby(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT']).rolling(window='360D', on='TIME_KEY', min_periods=1).TRANSACTION_ID_MASK \
                                                         .count().reset_index()['TRANSACTION_ID_MASK'].astype(int)

ml_dataset = create_aggregations(df_uniques, ml_dataset, '30_DAYS', 'CUSTSUBCAT_NUM_TRANSACTIONS', 'CUSTSUBCAT', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '90_DAYS', 'CUSTSUBCAT_NUM_TRANSACTIONS', 'CUSTSUBCAT', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '180_DAYS', 'CUSTSUBCAT_NUM_TRANSACTIONS', 'CUSTSUBCAT', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '360_DAYS', 'CUSTSUBCAT_NUM_TRANSACTIONS', 'CUSTSUBCAT', False)

In [23]:
# TOTAL QUATITY BOUGHT BY CUSTOMER BY SUBCATEGORY - CORRETO!

df['CUSTSUBCAT_TOTAL_QTY_BOUGHT_30_DAYS'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT']).rolling(window='30D', on='TIME_KEY', min_periods=1).QTY \
                                              .sum().reset_index()['QTY'].astype(int)

df['CUSTSUBCAT_TOTAL_QTY_BOUGHT_90_DAYS'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT']).rolling(window='90D', on='TIME_KEY', min_periods=1).QTY \
                                              .sum().reset_index()['QTY'].astype(int)

df['CUSTSUBCAT_TOTAL_QTY_BOUGHT_180_DAYS'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT']).rolling(window='180D', on='TIME_KEY', min_periods=1).QTY \
                                               .sum().reset_index()['QTY'].astype(int)

df['CUSTSUBCAT_TOTAL_QTY_BOUGHT_360_DAYS'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT']).rolling(window='360D', on='TIME_KEY', min_periods=1).QTY \
                                               .sum().reset_index()['QTY'].astype(int)


ml_dataset = create_aggregations(df, ml_dataset, '30_DAYS', 'CUSTSUBCAT_TOTAL_QTY_BOUGHT', 'CUSTSUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, '90_DAYS', 'CUSTSUBCAT_TOTAL_QTY_BOUGHT', 'CUSTSUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, '180_DAYS', 'CUSTSUBCAT_TOTAL_QTY_BOUGHT', 'CUSTSUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, '360_DAYS', 'CUSTSUBCAT_TOTAL_QTY_BOUGHT', 'CUSTSUBCAT', False)

In [24]:
# AVERAGE DAYS FOR NEXT CUSTOMER'S TRANSACTION WITH A CERTAIN SUBCATEGORY - CORRETO!

df_uniques = df.copy()
df_uniques['CUSTSUBCAT_DAYS_FOR_NEXT_TRANSACTION'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT'])['TIME_KEY'].diff(-1).dt.days \
                                              .fillna(0).apply(lambda x: abs(x))
df_uniques = df_uniques.drop_duplicates(subset='TRANSACTION_ID_MASK', keep='last').reset_index()

df_uniques['CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_30_DAYS'] = df_uniques.groupby(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT']) \
                                                                   .rolling(window='30D', on='TIME_KEY', min_periods=1).CUSTSUBCAT_DAYS_FOR_NEXT_TRANSACTION \
                                                                   .apply(lambda x: x[:-1].mean()).reset_index()['CUSTSUBCAT_DAYS_FOR_NEXT_TRANSACTION'] \
                                                                   .fillna(30)

df_uniques['CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_90_DAYS'] = df_uniques.groupby(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT']) \
                                                                   .rolling(window='90D', on='TIME_KEY', min_periods=1).CUSTSUBCAT_DAYS_FOR_NEXT_TRANSACTION \
                                                                   .apply(lambda x: x[:-1].mean()).reset_index()['CUSTSUBCAT_DAYS_FOR_NEXT_TRANSACTION'] \
                                                                   .fillna(60)

df_uniques['CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_180_DAYS'] = df_uniques.groupby(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT']) \
                                                                    .rolling(window='180D', on='TIME_KEY', min_periods=1).CUSTSUBCAT_DAYS_FOR_NEXT_TRANSACTION \
                                                                    .apply(lambda x: x[:-1].mean()).reset_index()['CUSTSUBCAT_DAYS_FOR_NEXT_TRANSACTION'] \
                                                                    .fillna(180)

df_uniques['CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION_360_DAYS'] = df_uniques.groupby(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT']) \
                                                                    .rolling(window='360D', on='TIME_KEY', min_periods=1).CUSTSUBCAT_DAYS_FOR_NEXT_TRANSACTION \
                                                                    .apply(lambda x: x[:-1].mean()).reset_index()['CUSTSUBCAT_DAYS_FOR_NEXT_TRANSACTION'] \
                                                                    .fillna(360)

ml_dataset = create_aggregations(df_uniques, ml_dataset, '30_DAYS', 'CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION', 'CUSTSUBCAT', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '90_DAYS', 'CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION', 'CUSTSUBCAT', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '180_DAYS', 'CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION', 'CUSTSUBCAT', False)
ml_dataset = create_aggregations(df_uniques, ml_dataset, '360_DAYS', 'CUSTSUBCAT_AVG_DAYS_FOR_NEXT_TRANSACTION', 'CUSTSUBCAT', False)

Other features

In [25]:
customer = fill_missing_values(df)  # Fills the missing values of the customer column

In [26]:
# create a dictionary that maps CUSTOMER_ACCOUNT_NR_MASK values to their corresponding gender values
customer_dict = dict(zip(customer['CUSTOMER_ACCOUNT_NR_MASK'], customer['GENDER']))

# map the gender values to the CUSTOMER_ACCOUNT_NR_MASK column in result_df using the customer_dict mapping
ml_dataset['GENDER'] = ml_dataset['CUSTOMER_ACCOUNT_NR_MASK'].map(customer_dict)

In [27]:
# create a dictionary that maps CUSTOMER_ACCOUNT_NR_MASK values to their corresponding gender values
family_dict = dict(zip(customer['CUSTOMER_ACCOUNT_NR_MASK'], customer['FAMILY_MEMBERS']))

# map the gender values to the CUSTOMER_ACCOUNT_NR_MASK column in result_df using the customer_dict mapping
ml_dataset['FAMILY_MEMBERS'] = ml_dataset['CUSTOMER_ACCOUNT_NR_MASK'].map(family_dict)

In [28]:
# create a dictionary that maps CUSTOMER_ACCOUNT_NR_MASK values to their corresponding gender values
lifestyle_dict = dict(zip(customer['CUSTOMER_ACCOUNT_NR_MASK'], customer['seg_lifestyle_cd']))

# map the gender values to the CUSTOMER_ACCOUNT_NR_MASK column in result_df using the customer_dict mapping
ml_dataset['SEG_LIFESTYLE_CD'] = ml_dataset['CUSTOMER_ACCOUNT_NR_MASK'].map(lifestyle_dict)

In [29]:
# create a dictionary that maps CUSTOMER_ACCOUNT_NR_MASK values to their corresponding gender values
lifestage_dict = dict(zip(customer['CUSTOMER_ACCOUNT_NR_MASK'], customer['seg_lifestage_cd']))

# map the gender values to the CUSTOMER_ACCOUNT_NR_MASK column in result_df using the customer_dict mapping
ml_dataset['SEG_LIFESTAGE_CD'] = ml_dataset['CUSTOMER_ACCOUNT_NR_MASK'].map(lifestage_dict)

In [30]:
query = client.query("""
   SELECT SUBCAT_CD_EXT, CAT_CD_EXT, PRICE_RANGE
   FROM tables_raw.dim_product
   """)

products = query.result().to_dataframe() # Wait for the job to complete.

In [31]:
# create a dictionary that maps CUSTOMER_ACCOUNT_NR_MASK values to their corresponding gender values
lifestage_dict = dict(zip(products['SUBCAT_CD_EXT'], products['CAT_CD_EXT']))

# map the gender values to the CUSTOMER_ACCOUNT_NR_MASK column in result_df using the customer_dict mapping
ml_dataset['CAT_CD_EXT'] = ml_dataset['SUBCAT_CD_EXT'].map(lifestage_dict)

Target features

In [37]:
ml_dataset = ml_dataset.sort_values(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT','YEAR','MONTH'])
ml_dataset = compute_target(ml_dataset)

# Load dataset into BigQuery

In [ ]:
# Shuffle the dataframe
ml_dataset = ml_dataset.sample(frac=1).reset_index(drop=True)

In [43]:
#### SAVE DATAFRAME TO BIGQUERY ####
client.load_table_from_dataframe(ml_dataset, 'tables_staging.df_models').result()

MemoryError: Unable to allocate 2.21 GiB for an array with shape (11, 26953560) and data type float64